# Launching Webservers
Webservers are useful platforms for deploying models.  Here we'll show some simple config for a webserver called "flask"

In [1]:
! cat hello_world.py

from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return 'HELLO WORLD'


Flask apps use the `@app.route` decorator to designate a handler for a path.  In this case, we're defining one for the root, or `/` path.

In [2]:
# ! export FLASK_APP=hello_world.py && flask run

Here's a slightly more complex one that permits a `POST` method to send larger amounts of data to the webserver.  We'll use this method to:
1. Send the input data payload
2. Deserialize the data payload into the raw data
3. Use the deserialized pipeline to make a new prediction from the raw data
4. Serialize the prediction
5. Send the serialized prediction back to the reqeuster

In [3]:
! cat app.py

from flask import Flask, request
import pandas as pd
import joblib
import json
import pipeline
from pipeline import *

app = Flask(__name__)

@app.route('/', methods = ['GET', 'POST'])
def index():
    if request.method == "POST":
        pipe = joblib.load("train_pipe.joblib")
        dat = pd.read_json(request.form["input"])
        return json.dumps(pipe.predict(dat).tolist())
    else:
        print("Remember to use the POST method for this server")

if __name__ == "__main__":
    app.run()


In [4]:
! python app.py

 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Mar/2021 17:01:38] "POST /prediction_echo HTTP/1.1" 405 -
^C
